In [22]:
!pip install xgboost


In [23]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import requests
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from datetime import datetime, timedelta

# Load ILI Data (historical weekly flu totals per state)
ili_file_path = "ILINet.csv"
ili_df = pd.read_csv(ili_file_path, na_values=["X"])

# Fetch Census Data from API (State-Level Population & Income)
census_url = "https://api.census.gov/data/2022/acs/acs1?get=NAME,B01003_001E,B19013_001E&for=state:*"
census_response = requests.get(census_url)
census_data = census_response.json()

# Convert to DataFrame
columns = ["State", "Population", "Median_Income", "State_Code"]
df_census = pd.DataFrame(census_data[1:], columns=columns)
df_census["Population"] = df_census["Population"].astype(int)
df_census["Median_Income"] = df_census["Median_Income"].astype(int)

# Merge ILI Data with Census Data
df = ili_df.merge(df_census, left_on="REGION", right_on="State", how="left")

# Keep relevant columns
df = df[["REGION", "YEAR", "WEEK", "ILITOTAL", "Population", "Median_Income"]]

# Convert YEAR & WEEK into DateTime format
df["DATE"] = pd.to_datetime(df["YEAR"].astype(str) + "-" + df["WEEK"].astype(str) + "-1", format="%Y-%W-%w")

# Drop YEAR & WEEK
df.drop(columns=["YEAR", "WEEK"], inplace=True)

# Sort by state & date
df.sort_values(by=["REGION", "DATE"], inplace=True)

# Display structure
print(df.head())


      REGION  ILITOTAL  Population  Median_Income       DATE
0    Alabama     249.0   5074296.0        59674.0 2010-10-04
52   Alabama     239.0   5074296.0        59674.0 2010-10-11
104  Alabama     232.0   5074296.0        59674.0 2010-10-18
156  Alabama     274.0   5074296.0        59674.0 2010-10-25
208  Alabama     342.0   5074296.0        59674.0 2010-11-01


In [24]:
# Function to create lag features
def create_lagged_features(df, lags=[1, 2, 3, 4]):
    for lag in lags:
        df[f"ILI_LAG_{lag}"] = df.groupby("REGION")["ILITOTAL"].shift(lag)
    return df

# Function to add rolling averages for trend detection
def create_rolling_features(df, window=4):
    df["ILI_ROLLING_MEAN"] = df.groupby("REGION")["ILITOTAL"].rolling(window=window).mean().reset_index(level=0, drop=True)
    return df

# Apply feature engineering
df = create_lagged_features(df)
df = create_rolling_features(df)

# Encode week of the year as a feature
df["WEEK"] = df["DATE"].dt.isocalendar().week


# Drop NaN rows from shifting
df.dropna(inplace=True)

# Show processed dataset
print(df.head())


      REGION  ILITOTAL  Population  Median_Income       DATE  ILI_LAG_1  \
208  Alabama     342.0   5074296.0        59674.0 2010-11-01      274.0   
260  Alabama     419.0   5074296.0        59674.0 2010-11-08      342.0   
312  Alabama     434.0   5074296.0        59674.0 2010-11-15      419.0   
364  Alabama     360.0   5074296.0        59674.0 2010-11-22      434.0   
416  Alabama     467.0   5074296.0        59674.0 2010-11-29      360.0   

     ILI_LAG_2  ILI_LAG_3  ILI_LAG_4  ILI_ROLLING_MEAN  WEEK  
208      232.0      239.0      249.0            271.75    44  
260      274.0      232.0      239.0            316.75    45  
312      342.0      274.0      232.0            367.25    46  
364      419.0      342.0      274.0            388.75    47  
416      434.0      419.0      342.0            420.00    48  


In [25]:
# Define features & target
X = df[["Population", "Median_Income", "ILI_LAG_1", "ILI_LAG_2", "ILI_LAG_3", "ILI_LAG_4", "ILI_ROLLING_MEAN", "WEEK"]]
y = df["ILITOTAL"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Train XGBoost Model (Using GPU)
xgb_model = xgb.XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=5,
    tree_method="hist",
    device="cuda",
    random_state=42
)

# Train the model
xgb_model.fit(X_train, y_train)

print("Model trained successfully!")


Model trained successfully!


In [26]:
xgb_model.set_params(device="cpu")  # Force model to run on CPU
y_pred = xgb_model.predict(X_test)  # Predict normally


In [27]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Make predictions on the test set
y_pred = xgb_model.predict(X_test)

# Calculate errors
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)  # Root Mean Squared Error
r2 = r2_score(y_test, y_pred)  # R-squared score

# Print results
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R² Score: {r2:.4f}")


Mean Absolute Error (MAE): 61.5878
Mean Squared Error (MSE): 48984.9085
Root Mean Squared Error (RMSE): 221.3253
R² Score: 0.9575


In [28]:
pip install optuna


Note: you may need to restart the kernel to use updated packages.


In [32]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Define objective function for Optuna
def objective(trial):
    # Suggest hyperparameters to optimize
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "tree_method": "hist",  # Use "hist" for faster training
        "device": "cuda"  # Enable GPU acceleration
    }

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Train model with suggested hyperparameters
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train)

    # Predict on test data
    y_pred = model.predict(X_test)

    # Calculate RMSE (lower is better)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    return rmse  # Optuna will try to minimize this

# Run Optuna optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)  # Adjust `n_trials` for longer tuning

# Show best hyperparameters
print("Best Hyperparameters:", study.best_params)


[I 2025-02-22 18:21:58,024] A new study created in memory with name: no-name-c8e6e151-380e-4ed9-86f6-596950ead8a5
[I 2025-02-22 18:21:58,300] Trial 0 finished with value: 221.52660092967355 and parameters: {'n_estimators': 300, 'learning_rate': 0.1975243962001115, 'max_depth': 5, 'subsample': 0.7414599633034853, 'colsample_bytree': 0.9475459104303356, 'gamma': 1.9885927398938719, 'min_child_weight': 10}. Best is trial 0 with value: 221.52660092967355.
[I 2025-02-22 18:21:58,407] Trial 1 finished with value: 244.87972948902828 and parameters: {'n_estimators': 100, 'learning_rate': 0.18500912860333496, 'max_depth': 5, 'subsample': 0.6886403272530262, 'colsample_bytree': 0.6163138675756376, 'gamma': 4.444915833614749, 'min_child_weight': 5}. Best is trial 0 with value: 221.52660092967355.
[I 2025-02-22 18:21:58,891] Trial 2 finished with value: 235.9228392271914 and parameters: {'n_estimators': 200, 'learning_rate': 0.10800390757516283, 'max_depth': 10, 'subsample': 0.9705337400252573, 'c

Best Hyperparameters: {'n_estimators': 900, 'learning_rate': 0.07571297597054956, 'max_depth': 7, 'subsample': 0.6707643820896211, 'colsample_bytree': 0.7918384490584499, 'gamma': 1.618739431660753, 'min_child_weight': 8}


In [33]:
# Get best parameters
best_params = study.best_params

# Train final model using best hyperparameters
xgb_model = xgb.XGBRegressor(**best_params)
xgb_model.fit(X_train, y_train)

# Save optimized model
xgb_model.save_model("ili_xgboost_optimized_optimized.json")

print("Optimized model trained & saved!")


Optimized model trained & saved!


In [34]:
# Predict using the optimized model
y_pred_opt = xgb_model.predict(X_test)

# Calculate accuracy metrics
mae = mean_absolute_error(y_test, y_pred_opt)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_opt))

print(f"Optimized Model MAE: {mae:.4f}")
print(f"Optimized Model RMSE: {rmse:.4f}")


Optimized Model MAE: 57.2974
Optimized Model RMSE: 213.6061
